# Evaluation


In [ ]:
library("caret")
library("rsample")
library("dplyr")
library("purrr")
library("ggplot2")
library("tidyr")

# simulation

In [ ]:
W <- 10
H <- 7

In [ ]:
f <- function(x) (x - pi)^2 / 10 + sin(x) / 5

In [ ]:
gen_data <- function(N) {
  x <- runif(N, 0, 2 * pi)
  y <- f(x)
  e <- rnorm(N, 0, 1 / 10)
  y <- y + e
  all_data <- tibble(data.frame(x = x, y = y))
  return(all_data)
}

N_train <- 250
K_max <- 55
Num_K <- min(K_max,100)


all_data <- gen_data(N_train)

K_seq <- floor(seq(1, K_max, length.out = min(Num_K, K_max)))
K_seq

options(repr.plot.width = W, repr.plot.height = H, repr.plot.res = 100)
ggplot(data = all_data, mapping = aes(x = x, y = y)) +
  geom_point()

In [ ]:
super_test <- gen_data(5000) # wouldn't have this in real life, but good for demo purposes

# Simple test/train split

In [ ]:
splt <- initial_split(all_data, prop = .9)
splt

In [ ]:
training(splt) %>% dim()
testing(splt) %>% dim()

In [ ]:
# build on the testing data
knn_mod <- knnreg(y ~ ., data = training(splt), k = 5)

In [ ]:
calc_metric <- function(y, yhat) sqrt(mean((y - yhat)^2))

In [ ]:
train_preds <- predict(knn_mod, training(splt))
MSE_train <- calc_metric(training(splt) %>% pull(y), train_preds)
MSE_train

In [ ]:
test_preds <- predict(knn_mod, testing(splt))
MSE_test <- calc_metric(testing(splt) %>% pull(y), test_preds)
MSE_test

In [ ]:
plot(all_data$x, all_data$y)
xe <- data.frame(x = sort(runif(1000, 0, 2 * pi)))
lines(xe$x, f(xe$x), col = "blue", lwd = 5)
lines(xe$x, predict(knn_mod, xe), col = "red", lwd = 5)

notice how the training MSE is typically lower than the testing MSE

In [ ]:
# demo only (wouldn't have in real analysis)
super_preds <- predict(knn_mod, super_test)
MSE_super <- calc_metric(super_test %>% pull(y), super_preds)
MSE_super

In [ ]:
over_K = function(K){
    knn_mod <- knnreg(y ~ ., data = training(splt), k = K)

    train_preds <- predict(knn_mod, training(splt))
    MSE_train <- calc_metric(training(splt) %>% pull(y), train_preds)
    MSE_train

    test_preds <- predict(knn_mod, testing(splt))
    MSE_test <- calc_metric(testing(splt) %>% pull(y), test_preds)
    MSE_test

    super_preds <- predict(knn_mod, super_test)
    MSE_super <- calc_metric(super_test %>% pull(y), super_preds)
    MSE_super

    return(tibble(train=MSE_train,test=MSE_test,super=MSE_super,K=K))
}

In [ ]:
ret <- map(K_seq, ~ over_K(.x))

In [ ]:
tttbl <- ret %>% bind_rows()

In [ ]:
tttbl %>% pivot_longer(cols=c(train,test,super)) %>% 
    ggplot(mapping=aes(x=1/K,y=value,color=name,group=name))+geom_point() + 
    scale_x_log10() + 
    scale_y_log10()

In [ ]:
tttbl %>% filter(test==min(test))

A little simulation:

In [ ]:
sim = function(i){
    splt <- initial_split(all_data, prop = .9)

    over_K = function(K){
        knn_mod <- knnreg(y ~ ., data = training(splt), k = K)
    
        train_preds <- predict(knn_mod, training(splt))
        MSE_train <- calc_metric(training(splt) %>% pull(y), train_preds)
        MSE_train
    
        test_preds <- predict(knn_mod, testing(splt))
        MSE_test <- calc_metric(testing(splt) %>% pull(y), test_preds)
        MSE_test
    
        super_preds <- predict(knn_mod, super_test)
        MSE_super <- calc_metric(super_test %>% pull(y), super_preds)
        MSE_super
    
        return(tibble(train=MSE_train,test=MSE_test,super=MSE_super,K=K))
    }

    ret <- map(K_seq, ~ over_K(.x))
    tttbl <- ret %>% bind_rows()
    return(tttbl %>% filter(test==min(test)))
}

In [ ]:
sim_out <- map(1:10,sim)

In [ ]:
sim_tbl <- sim_out %>% bind_rows()
sim_tbl$i <- 1:nrow(sim_tbl)
sim_tbl

In [ ]:
sim_tbl %>% pivot_longer(cols=c(train,test,super)) %>% 
    ggplot(mapping=aes(x=i,y=value,color=name,shape=name)) + 
    geom_point()

# Cross-validation

In [ ]:
xvsplt <- vfold_cv(all_data, v = 10)

In [ ]:
map(xvsplt$splits, dim)

In [ ]:
training(xvsplt$splits[[2]]) %>% dim()
testing(xvsplt$splits[[2]]) %>% dim()

let's put this in a function

In [ ]:
split_eval <- function(splt) {
  knn_mod <- knnreg(y ~ ., data = training(splt), k = 5)

  train_preds <- predict(knn_mod, training(splt))
  MSE_train <- calc_metric(training(splt) %>% pull(y), train_preds)

  test_preds <- predict(knn_mod, testing(splt))
  MSE_test <- calc_metric(testing(splt) %>% pull(y), test_preds)

  return(data.frame(
    train = MSE_train,
    test = MSE_test
  ))
}

In [ ]:
split_eval(xvsplt$splits[[1]])

In [ ]:
split_eval(xvsplt$splits[[2]])

In [ ]:
xv_res <- map(xvsplt$splits, split_eval) %>% bind_rows()
xv_res$fold <- 1:length(xvsplt$splits)
xv_res

In [ ]:
xv_res_long <- xv_res %>% pivot_longer(cols = c(train, test))
xv_res_long

In [ ]:
ggplot(data = xv_res_long, mapping = aes(x = fold, y = value, color = name)) +
  geom_point() +
  geom_hline(yintercept = MSE_super, lty = 2)

in total summary we can summarize the RMSEs

In [ ]:
xv_res_long %>%
  group_by(name) %>%
  summarize(mean = mean(value), sd = sd(value))

In [ ]:
# demo only (wouldn't have in real analysis)
super_preds <- predict(knn_mod, super_test)
MSE_super <- calc_metric(super_test %>% pull(y), super_preds)
MSE_super

# Train/Test/Validate to choose $K$

How can we use this to choose a value of $k$ for KNN? Use a train/validate/test 3-way split

In [ ]:
splt <- initial_validation_split(all_data, prop = c(0.6, .2))
splt

In [ ]:
split_eval <- function(splt, K = NULL) {
  knn_mod <- knnreg(y ~ ., data = training(splt), k = K)

  train_preds <- predict(knn_mod, training(splt))
  MSE_train <- calc_metric(training(splt) %>% pull(y), train_preds)

  val_preds <- predict(knn_mod, validation(splt))
  MSE_val <- calc_metric(validation(splt) %>% pull(y), val_preds)

  test_preds <- predict(knn_mod, testing(splt))
  MSE_test <- calc_metric(testing(splt) %>% pull(y), test_preds)

  super_preds <- predict(knn_mod, super_test)
  MSE_super <- calc_metric(super_test %>% pull(y), super_preds)
  MSE_super

  return(data.frame(
    train = MSE_train,
    test = MSE_test,
    val = MSE_val,
    super = MSE_super,
    K = K
  ))
}

In [ ]:
split_eval(splt, K = 5)

In [ ]:
ttv_res <- map(K_seq, ~ split_eval(splt, K = .x))

In [ ]:
ttv_tbl <- ttv_res %>% bind_rows()
ttv_tbl %>% head()

In [ ]:
ttv_long <- ttv_tbl %>% pivot_longer(cols = c(train, val, super))
ttv_long %>% head()

In [ ]:
ggplot(data = ttv_long, mapping = aes(x = K, y = value, color = name)) +
  geom_point() +
  scale_x_log10() +
  scale_y_log10()

In [ ]:
ggplot(data = ttv_long, mapping = aes(x = 1 / K, y = value, color = name)) +
  geom_point() +
  scale_x_log10() +
  scale_y_log10()

In [ ]:
ttv_tbl %>% filter(val == min(val))

In [ ]:
K_hat <- ttv_tbl %>%
  filter(val == min(val)) %>%
  pull(K)
K_hat

In [ ]:
knn_mod <- knnreg(y ~ ., data = all_data, k = K_hat)
plot(all_data$x, all_data$y)
xe <- data.frame(x = sort(runif(1000, 0, 2 * pi)))
lines(xe$x, f(xe$x), col = "blue", lwd = 5)
lines(xe$x, predict(knn_mod, xe), col = "red", lwd = 5)

# Cross Valdation to choose $K$

In [ ]:
split_eval_K <- function(splt, K) {
  knn_mod <- knnreg(y ~ ., data = training(splt), k = K)

  train_preds <- predict(knn_mod, training(splt))
  MSE_train <- calc_metric(training(splt) %>% pull(y), train_preds)

  test_preds <- predict(knn_mod, testing(splt))
  MSE_test <- calc_metric(testing(splt) %>% pull(y), test_preds)

  return(data.frame(
    train = MSE_train,
    test = MSE_test,
    K = K
  ))
}

split_eval <- function(splt) {
  K_seq <- floor(seq(1, K_max, length.out = 50))
  ret <- map(K_seq, ~ split_eval_K(splt, .x))
  return(ret)
}

In [ ]:
split_eval_i <- function(i) {
  sei <- split_eval(xvsplt$splits[[i]]) %>% bind_rows()
  sei$i <- i
  return(sei)
}

res <- map(seq_along(xvsplt$splits), split_eval_i) %>% bind_rows()

In [ ]:
res %>% sample_n(10)

In [ ]:
plt_df <- res %>%
  pivot_longer(cols = c(train, test)) %>%
  group_by(K, name) %>%
  summarize(mean = mean(value), q25 = quantile(value, .25), q75 = quantile(value, .75)) %>%
  ungroup()
plt_df %>% head()

In [ ]:
plt_df %>% ggplot(mapping = aes(x = K, y = mean, group = name, color = name, fill = name)) +
  geom_point() +
  geom_ribbon(mapping = aes(ymin = q25, ymax = q75), alpha = 0.25) +
  scale_x_log10() +
  scale_y_log10()

In [ ]:
plt_df %>%
  filter(name == "test") %>%
  filter(mean == min(mean))

In [ ]:
plt_df %>%
  ggplot(mapping = aes(x = 1 / K, y = mean, group = name, color = name, fill = name)) +
  geom_point() +
  geom_ribbon(mapping = aes(ymin = q25, ymax = q75), alpha = 0.25) +
  scale_x_log10() +
  scale_y_log10()

In [ ]:
K_hat <- plt_df %>%
  filter(name == "test") %>%
  filter(mean == min(mean)) %>%
  pull(K)
K_hat
knn_mod <- knnreg(y ~ ., data = all_data, k = K_hat)
plot(all_data$x, all_data$y)
xe <- data.frame(x = sort(runif(1000, 0, 2 * pi)))
lines(xe$x, f(xe$x), col = "blue", lwd = 5)
lines(xe$x, predict(knn_mod, xe), col = "red", lwd = 5)

# Nested x-validation

In [ ]:
# outer loop = split into test and trainval datasets
# inner loop = split into train/val and search over k

In [ ]:
ncvsplt <- nested_cv(all_data, outside = vfold_cv(v = 10), inside = vfold_cv(v = 10))

In [ ]:
print(ncvsplt)

In [ ]:
ncvsplt$splits

In [ ]:
outer_splt <- ncvsplt$splits[[1]]
training(outer_splt) %>% dim()
testing(outer_splt) %>% dim()

In [ ]:
ncvsplt$inner_resamples[[1]]$splits

In [ ]:
inner_splt <- ncvsplt$inner_resamples[[3]]$splits[[1]]
analysis(inner_splt) %>% dim()
assessment(inner_splt) %>% dim()

Copy prev code and put in function called `build_mdl`

In [ ]:
build_mdl <- function(splt, K_seq = floor(seq(1, K_max, length.out = 50))) {
  inner_eval <- function(splt, K = NULL) {
    knn_mod <- knnreg(y ~ ., data = training(splt), k = K)

    train_preds <- predict(knn_mod, training(splt))
    MSE_train <- calc_metric(analysis(splt) %>% pull(y), train_preds)

    val_preds <- predict(knn_mod, testing(splt))
    MSE_val <- calc_metric(testing(splt) %>% pull(y), val_preds)

    return(data.frame(
      train = MSE_train,
      val = MSE_val,
      K = K
    ))
  }

  tv_res <- map(K_seq, ~ inner_eval(splt, K = .x))
  tv_tbl <- tv_res %>% bind_rows()
  return(tv_tbl)
}

In [ ]:
i <- 1
inner_splits_i <- ncvsplt$inner_resamples[[i]]$splits
outer_splt_i <- ncvsplt$splits[[i]]

In [ ]:
inner_eval_j <- map(seq_along(inner_splits_i), function(j) {
  tmp <- build_mdl(inner_splits_i[[j]])
  tmp$j <- j
  return(tmp)
})

In [ ]:
inner_eval <- inner_eval_j %>% bind_rows()
inner_eval %>% sample_n(10)

In [ ]:
inner_smry <- inner_eval %>%
  pivot_longer(cols = c(train, val)) %>%
  group_by(K, name) %>%
  summarize(mean = mean(value), q25 = quantile(value, .25), q75 = quantile(value, .75)) %>%
  ungroup()
inner_smry %>% sample_n(10)

In [ ]:
ggplot(data = inner_smry, mapping = aes(x = K, y = mean, fill = name, color = name, group = name)) +
  geom_point() +
  geom_ribbon(mapping = aes(ymin = q25, ymax = q75), alpha = .25) +
  scale_y_log10() +
  scale_x_log10()

In [ ]:
K_hat <- inner_smry %>%
  filter(name == "val") %>%
  filter(mean == min(mean)) %>%
  pull(K)
K_hat

In [ ]:
K_hat_mdl <- knnreg(y ~ ., data = training(outer_splt_i), k = K_hat)
test_preds <- predict(K_hat_mdl, testing(outer_splt_i))
MSE_test <- calc_metric(testing(outer_splt_i) %>% pull(y), test_preds)
MSE_test

In [ ]:
super_preds <- predict(K_hat_mdl, super_test)
MSE_super <- calc_metric(super_test %>% pull(y), super_preds)
MSE_super

Ok, now we wrap this all up in a function

In [ ]:
outer_eval <- function(i) {
  outer_splt_i <- ncvsplt$splits[[i]]
  inner_splits_i <- ncvsplt$inner_resamples[[i]]$splits

  # my model building procedure
  inner_eval_j <- map(seq_along(inner_splits_i), function(j) {
    tmp <- build_mdl(inner_splits_i[[j]])
    tmp$j <- j
    return(tmp)
  })

  inner_eval <- inner_eval_j %>% bind_rows()

  inner_smry <- inner_eval %>%
    pivot_longer(cols = c(train, val)) %>%
    group_by(K, name) %>%
    summarize(mean = mean(value), q25 = quantile(value, .25), q75 = quantile(value, .75)) %>%
    ungroup()

  K_hat <- inner_smry %>%
    filter(name == "val") %>%
    filter(mean == min(mean)) %>%
    pull(K)

  inner_val <- inner_smry %>%
    filter(name == "val") %>%
    filter(K == K_hat) %>%
    pull(mean)

  inner_train <- inner_smry %>%
    filter(name == "train") %>%
    filter(K == K_hat) %>%
    pull(mean)


  K_hat_mdl <- knnreg(y ~ ., data = training(outer_splt_i), k = K_hat)
  # end my model building procedure

  test_preds <- predict(K_hat_mdl, testing(outer_splt_i))
  MSE_test <- calc_metric(testing(outer_splt_i) %>% pull(y), test_preds)

  super_preds <- predict(K_hat_mdl, super_test)
  MSE_super <- calc_metric(super_test %>% pull(y), super_preds)
  MSE_super

  return(tibble(test = MSE_test, val = inner_val, train = inner_train, super = MSE_super, K_hat = K_hat))
}

In [ ]:
outer_eval <- map(seq_along(ncvsplt$splits), outer_eval)

In [ ]:
outer_df <- outer_eval %>% bind_rows()
outer_df$fold <- 1:nrow(outer_df)
outer_df

In [ ]:
outer_longer <- outer_df %>% pivot_longer(cols = c(test, val, train, super))
outer_longer

In [ ]:
ggplot(data = outer_longer, mapping = aes(x = fold, y = value, color = name, shape = name)) +
  geom_point(size = 5)

In [ ]:
outer_longer %>% head()

In [ ]:
outer_longer %>%
  group_by(name) %>%
  summarize(mean = mean(value), sd = sd(value))